In [104]:
#Update your token
STUDENT_TOKEN = 'EUGENIOMARCHIORI'

Server communication game and game_loop.

In [105]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    print(self.state)
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Random solver

In [106]:
def random_solver(game):
  actions = game.actions()
  return actions[0]

Game specific code

In [107]:
from functools import reduce
from copy import copy, deepcopy
import json


class Brisca(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [i for i in range(len(self.state['hand']))]

  def card_str(self, card):
    return '+'.join(map(str, card))

  def print_game(self):
    print('Hand: ' + ' '.join(map(self.card_str, self.state['hand'])) + '  Table: ' + ' '.join(map(self.card_str, self.state['table'])) + '  Trump: ' + self.card_str(self.state['trump']))
    print('Score X: ' + str(self.state['score'][0]) + '  Score O:' + str(self.state['score'][1]))

  def other_player(self):
    if self.player == 'X': return 'O'
    if self.player == 'O': return 'X'

In [108]:
game_loop(random_solver, Brisca, 'brisca', multi_player=False, id=None)

Creating new game...
Joining game with id: 129
Playing as O
Waiting for the other player to join...
Hand: 4+D 10+H 10+C  Table: 3+S  Trump: 6+S
Score X: 0  Score O:0
Making next move...
b'Valid move'
Hand: 10+H 10+C 2+S  Table: 3+H  Trump: 6+S
Score X: 10  Score O:0
Making next move...
b'Valid move'
Hand: 10+C 2+S 7+S  Table: 11+S  Trump: 6+S
Score X: 22  Score O:0
Making next move...
b'Valid move'
Hand: 2+S 7+S 5+C  Table: 12+C  Trump: 6+S
Score X: 27  Score O:0
Making next move...
b'Valid move'
Hand: 7+S 5+C 6+C  Table:   Trump: 6+S
Score X: 27  Score O:4
Making next move...
b'Valid move'
Hand: 5+C 6+C 5+H  Table:   Trump: 6+S
Score X: 27  Score O:4
Making next move...
b'Valid move'
Hand: 6+C 5+H 8+H  Table:   Trump: 6+S
Score X: 27  Score O:4
Making next move...
b'Valid move'
Hand: 5+H 8+H 6+H  Table:   Trump: 6+S
Score X: 27  Score O:4
Making next move...
b'Valid move'
Hand: 8+H 6+H 4+S  Table:   Trump: 6+S
Score X: 27  Score O:7
Making next move...
b'Valid move'
Hand: 6+H 4+S 9+S 